<a href="https://colab.research.google.com/github/OlgaZy/Python_for_DA/blob/main/HW_15_4_%D0%90%D0%BD%D0%B0%D0%BB%D1%96%D0%B7_%D0%90_%D0%92_%D1%82%D0%B5%D1%81%D1%82%D1%96%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

**Завдання 1.**

Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.stats.api as sms

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/DA_in_IT/Data/cookie_cats.csv')
df

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [5]:
effect_size = sms.proportion_effectsize(0.20, 0.19)

required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1
    )                                                  # Розрахунок необхідного розміру вибірки


print(round(required_n))

24638


**Висновок:** Нам буде потрібно мінімум 24638 спостережень для кожної групи.


Формування цільового формату даних

In [6]:
df["retention_7"] = df["retention_7"].astype(int)
df.head(5)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,0
1,337,gate_30,38,True,0
2,377,gate_40,165,True,0
3,483,gate_40,1,False,0
4,488,gate_40,179,True,1


**Завдання 2.**

Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [7]:
retention_7_mean= df.groupby('version')['retention_7'].mean()
retention_7_mean

,retention_7
version,
gate_30,0.190201
gate_40,0.182000


**Формулювання гіпотези**

H₀: 𝑝𝐴=𝑝𝐵  - різниці в утриманні на 7-й день між версіями gate_30 і gate_40 немає.

H₁:𝑝𝐴≠𝑝𝐵  - одна з версій (наприклад, gate_40) має краще утримання на 7-й день.

Це двосторонній тест.

**Завдання 3.**

Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [8]:
# Підрахунок кількості успіхів та розмірів вибірки
grouped = df.groupby("version")["retention_7"].agg(["sum", "count"])
print(grouped)

retention_1, n1 = grouped.loc["gate_30"]
retention_2, n2 = grouped.loc["gate_40"]

          sum  count
version             
gate_30  8502  44700
gate_40  8279  45489


In [9]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
# n1 - control_results = ab_test[ab_test['group'] == 'control']['converted']
# n2 - treatment_results = ab_test[ab_test['group'] == 'treatment']['converted']

# retention_1 - n_con = control_results.count()
# retention_1 - n_treat = treatment_results.count()

successes = [retention_1, retention_2]
nobs = [n1, n2]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
#z_stat, pval = proportions_ztest(successes, nobs=nobs, alternative='larger') для правостороннього тесту = H₁:𝑝𝐴<𝑝𝐵


(lower_a, lower_b), (upper_a, upper_b) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'Довірчий інтервал 95% для групи A: [{lower_a:.3f}, {upper_a:.3f}]')
print(f'Довірчий інтервал 95% для групи B: [{lower_b:.3f}, {upper_b:.3f}]')
print(f"=====================================")
alpha=0.05

if pval < alpha:
    print(f"Відхиляємо H0: є різниця в показниках конверсії на 7-й день між версіями gate_30 і gate_40 на рівні значущості {alpha}")
else:
    print(f"Не відхиляємо H0: суттєвої різниці в конверсіях немає на рівні значущості {alpha}")

z statistic: 3.16
p-value: 0.002
Довірчий інтервал 95% для групи A: [0.187, 0.194]
Довірчий інтервал 95% для групи B: [0.178, 0.186]
Відхиляємо H0: є різниця в показниках конверсії на 7-й день між версіями gate_30 і gate_40 на рівні значущості 0.05


**Bисновки:**

Оскільки p-значення (0.002) менше рівня значущості 0.05, нульову гіпотезу
відхиляємо. Це означає, що існує статистично значуща різниця між показниками конверсії двох груп на 7-й день.

Довірчі інтервали для пропорцій у групах не перетинаються, що додатково підтримує висновок про їх різницю.


**Завдання 4.**

Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


**Формулювання гіпотези**

H₀: Дві категоріальні змінні є незалежними. Тобто конверсія однакова для обох версій розміщення gate.

H₁: Існує зв’язок між змінними, тобто версія воріт впливає на показник утримання, і частки успішних утримань у двох версіях відрізняються.


In [31]:
grouped = df.groupby("version")["retention_7"].agg(["sum", "count"])
grouped['not_successes'] = grouped['count'] - grouped['sum']
grouped

,sum,count,not_successes
version,,,
gate_30,8502,44700,36198
gate_40,8279,45489,37210


In [30]:
# Формуємо таблицю спостережень
observed = np.array([[8502, 36198],
                     [8279, 37210]])

chi2, p, dof, expected = stats.chi2_contingency(observed)

print(f"χ² = {chi2:.3f}")
print(f"p-value = {p:.5f}")
print(f"Ступені свободи = {dof}")
print("Очікувані частоти:\n", expected)

χ² = 9.959
p-value = 0.00160
Ступені свободи = 1
Очікувані частоти:
 [[ 8317.09742873 36382.90257127]
 [ 8463.90257127 37025.09742873]]


**Висновоки: **

1. Оскільки  𝑝=0.00160 < 0.05 , відхиляємо  𝐻0 . → Є статистично значуща різниця у конверсії між групами A та B. Тобто конверсія залежить від версії розміщення gate.

2. Значення статистики χ² відображає ступінь відхилення спостережених даних від очікуваних при нульовій гіпотезі,
